# Enviroweather Private Weather Stations Python package: `ewx_pws`
## Example usage

Using this package in a project to pull weather data you start as follows

In [1]:

from ewx_pws import __version__, ewx_pws
print(__version__)

0.1.0


This package works with the following station types:

In [2]:
print(ewx_pws.STATION_TYPE_LIST)

['ZENTRA', 'ONSET', 'DAVIS', 'RAINWISE', 'SPECTRUM', 'LOCOMOS', 'GENERIC']


The main function accepts configuration information for one or more weather stations API (clouds), 
and uses that to connect to the API for that station and pull data for a specific time interval. 

The configuration can be saved in a CSV file or in the future in a database table. 

This example usage notebook assumes the values are in a CSV file named 'test_stations.csv'  Alter the variable for this file name
to use a different file. 

The file must be in the format :  


In [3]:
# this file must be present for the rest of the notebook to work.  You can create this file with any name, 
# but must be loaded here
station_config_file = "../test_stations.csv"

In [4]:
# load weather station configuration from an environment file
stations = ewx_pws.station_dict_from_file(station_config_file)

print("full station config:")
for station in stations.values():
    print(station.config)

2023-08-23 10:48:59,308-49919-DEBUG-instantiating EWXZENTRA01
2023-08-23 10:48:59,309-49919-DEBUG-instantiating EWXDAVIS01
2023-08-23 10:48:59,309-49919-DEBUG-instantiating EWXSPECTRUM01
2023-08-23 10:48:59,309-49919-DEBUG-instantiating EWXONSET01
2023-08-23 10:48:59,310-49919-DEBUG-instantiating EWXRAINWISE01
2023-08-23 10:48:59,310-49919-DEBUG-instantiating EWXLOCOMOS06


full station config:
station_id='EWXZENTRA01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='ZENTRA' tz='ET' sn='z6-12564' token='5b8f637a718b9e3ad6e31eec81f0b51d91ac38bd'
station_id='EWXDAVIS01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='DAVIS' tz='ET' sn='117005' apikey='aoqdcbirudd1sarq6erfj6tgtw67sbvo' apisec='elvr77yhdx3p0286cwnnqqstdwqg8zxf'
station_id='EWXSPECTRUM01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='SPECTRUM' tz='ET' sn='50400123' apikey='11a5c3a939856b08677b7a072f8e6865'
station_id='EWXONSET01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='ONSET' tz='ET' sn='21092695' client_id='Enviroweather_WS' client_secret='75d2b7f58f5d0cac699f5b9616318be268057de6' ret_form='JSON' user_id='12848' sensor_sn={'atemp': '21079936-1', 'pcpn': '21085496-1', 'relh': '21079936-2'}
station_id='EWXRAINWISE01' install_date=datetime.datetime(2023, 5, 1, 0, 0) station_type='RAINWISE' tz='ET' username='200000000500' sid='6

In [5]:
print("stations in config file:")
for station in stations.values():
    print('----------')
    print('id:      ', station.id)
    print('type:    ', station.config.station_type)
    print('timezone:', station.config.tz)


stations in config file:
----------
id:       EWXZENTRA01
type:     ZENTRA
timezone: ET
----------
id:       EWXDAVIS01
type:     DAVIS
timezone: ET
----------
id:       EWXSPECTRUM01
type:     SPECTRUM
timezone: ET
----------
id:       EWXONSET01
type:     ONSET
timezone: ET
----------
id:       EWXRAINWISE01
type:     RAINWISE
timezone: ET
----------
id:       EWXLOCOMOS06
type:     LOCOMOS
timezone: ET


Get sample weather without sending an interval of time, which uses the default: the previous 15 minute period (e.g. if it's 1:17, then from 1:00 to 1:15)

In [11]:
# get weather for these stations, leveraging the MultiweatherAPI python package
# pick one station from the list
station = list(stations.values())[1]
wdata = station.get_readings()
print(wdata)


2023-08-23 10:50:06,720-49919-DEBUG-Starting new HTTPS connection (1): api.weatherlink.com:443
2023-08-23 10:50:06,926-49919-DEBUG-https://api.weatherlink.com:443 "GET /v2/historic/117005?api-key=aoqdcbirudd1sarq6erfj6tgtw67sbvo&t=1692802206&start-timestamp=1692801000&end-timestamp=1692801900&api-signature=cf1e3ed0fa90379de60706e98cbb8a688eb03126eb9b3e4917987efef6daf8ff HTTP/1.1" 200 5087


station_id='EWXDAVIS01' station_type='DAVIS' request_id='89e9db50-10bb-4caf-b7e4-761742d31c7a' request_datetime=datetime.datetime(2023, 8, 23, 18, 50, 6, 573670, tzinfo=datetime.timezone.utc) time_interval=UTCInterval(start=datetime.datetime(2023, 8, 23, 14, 30, tzinfo=datetime.timezone.utc), end=datetime.datetime(2023, 8, 23, 14, 45, tzinfo=datetime.timezone.utc)) package_version='0.1' responses=[WeatherAPIResponse(url='https://api.weatherlink.com/v2/historic/117005?api-key=aoqdcbirudd1sarq6erfj6tgtw67sbvo&t=1692802206&start-timestamp=1692801000&end-timestamp=1692801900&api-signature=cf1e3ed0fa90379de60706e98cbb8a688eb03126eb9b3e4917987efef6daf8ff', status_code='200', reason='OK', text='{"station_id_uuid":"ed88c104-36a4-4244-ad3f-7f048dfa5a50","sensors":[{"lsid":434004,"data":[{"noise_floor_rssi":-83,"port_energy_consumed_4":0,"port_energy_consumed_3":0,"port_energy_consumed_2":0,"bluetooth_firmware_version":null,"beacon_interval":2097,"port_energy_consumed_1":0,"door_switch_status":0

The output uses a class WeatherReading which has metadata, and then a list of responses from the API.  Some APIs require multiple calls to the API for periods > 24 hours, so the responses are always stored in a list

In [12]:
output_from_api = wdata.responses[0]
print(output_from_api)  # response object from requests lib
print("this is the 'text' usually in json format, from the station")
# see the 'requests' doc for more about how to use the response object (link).  
print(output_from_api.text)


url='https://api.weatherlink.com/v2/historic/117005?api-key=aoqdcbirudd1sarq6erfj6tgtw67sbvo&t=1692802206&start-timestamp=1692801000&end-timestamp=1692801900&api-signature=cf1e3ed0fa90379de60706e98cbb8a688eb03126eb9b3e4917987efef6daf8ff' status_code='200' reason='OK' text='{"station_id_uuid":"ed88c104-36a4-4244-ad3f-7f048dfa5a50","sensors":[{"lsid":434004,"data":[{"noise_floor_rssi":-83,"port_energy_consumed_4":0,"port_energy_consumed_3":0,"port_energy_consumed_2":0,"bluetooth_firmware_version":null,"beacon_interval":2097,"port_energy_consumed_1":0,"door_switch_status":0,"link_layer_packets_received":161,"solar_panel_voltage":1490,"rank":576,"health_version":1,"false_wakeup_rssi":-72,"inside_box_temp":75.32349,"dcell_battery_voltage":5260,"power_percentage_tx":0,"last_rx_rssi":-33,"power_percentage_mcu":5,"application_firmware_version":1620422234,"false_wakeup_count":673,"etx":313,"rpl_mode":0,"uptime":48976515,"number_of_neighbors":1,"last_parent_rtt_ping":0,"li_ion_cell_voltage":4033

However, we probably want to specify the timer interval, so there is a function for creating that.   The station classes only accept timestamps that have a timezone, and that timezone must be UTC.   Most non-UTC timezones are affected by daylight svings, but now UtC.  


In [13]:
from ewx_pws.time_intervals import UTCInterval
# get an hour interval: 
interval = UTCInterval.previous_interval(delta_mins = 60)
# use that to get data
wdata = station.get_readings(interval.start, interval.end)
print(wdata)

2023-08-23 10:50:10,902-49919-DEBUG-Starting new HTTPS connection (1): api.weatherlink.com:443
2023-08-23 10:50:11,160-49919-DEBUG-https://api.weatherlink.com:443 "GET /v2/historic/117005?api-key=aoqdcbirudd1sarq6erfj6tgtw67sbvo&t=1692802210&start-timestamp=1692798300&end-timestamp=1692801900&api-signature=b81f2465e4fad0cce81e7dbf5b2581d68f101d2c1b350f771390f32c10fd7714 HTTP/1.1" 200 None


station_id='EWXDAVIS01' station_type='DAVIS' request_id='a7410253-583c-4204-96bb-77bf6a2a14e7' request_datetime=datetime.datetime(2023, 8, 23, 18, 50, 10, 899160, tzinfo=datetime.timezone.utc) time_interval=UTCInterval(start=datetime.datetime(2023, 8, 23, 13, 45, tzinfo=datetime.timezone.utc), end=datetime.datetime(2023, 8, 23, 14, 45, tzinfo=datetime.timezone.utc)) package_version='0.1' responses=[WeatherAPIResponse(url='https://api.weatherlink.com/v2/historic/117005?api-key=aoqdcbirudd1sarq6erfj6tgtw67sbvo&t=1692802210&start-timestamp=1692798300&end-timestamp=1692801900&api-signature=b81f2465e4fad0cce81e7dbf5b2581d68f101d2c1b350f771390f32c10fd7714', status_code='200', reason='OK', text='{"station_id_uuid":"ed88c104-36a4-4244-ad3f-7f048dfa5a50","sensors":[{"lsid":434004,"data":[{"noise_floor_rssi":-109,"port_energy_consumed_4":0,"port_energy_consumed_3":0,"port_energy_consumed_2":0,"bluetooth_firmware_version":null,"beacon_interval":2097,"port_energy_consumed_1":0,"door_switch_status"

The output from the api is a mix of the JSON output from the stations and metadata about how the request for data was made.   Each station has it's own format from the API, and we want to transform that to a standardize format for EWX. 

The station object stores the most recent request, and can run the `transform` method without an argument, and it will to use the latest api request, or you can send data. 



In [16]:
from pprint import pprint
readings = station.transform(wdata)
for reading in readings:
    pprint(reading)

2023-08-23 10:50:53,566-49919-DEBUG-transformed_reading type <class 'list'>: [{'data_datetime': datetime.datetime(2023, 8, 23, 17, 50, tzinfo=datetime.timezone.utc), 'atemp': 940443648.89, 'pcpn': 0.0, 'relh': 80}, {'data_datetime': datetime.datetime(2023, 8, 23, 17, 55, tzinfo=datetime.timezone.utc), 'atemp': 940443815.56, 'pcpn': 0.0, 'relh': 79}, {'data_datetime': datetime.datetime(2023, 8, 23, 18, 0, tzinfo=datetime.timezone.utc), 'atemp': 940443982.22, 'pcpn': 0.0, 'relh': 79}, {'data_datetime': datetime.datetime(2023, 8, 23, 18, 5, tzinfo=datetime.timezone.utc), 'atemp': 940444148.89, 'pcpn': 0.0, 'relh': 78}, {'data_datetime': datetime.datetime(2023, 8, 23, 18, 10, tzinfo=datetime.timezone.utc), 'atemp': 940444315.56, 'pcpn': 0.0, 'relh': 79}, {'data_datetime': datetime.datetime(2023, 8, 23, 18, 15, tzinfo=datetime.timezone.utc), 'atemp': 940444482.22, 'pcpn': 0.0, 'relh': 79}, {'data_datetime': datetime.datetime(2023, 8, 23, 18, 20, tzinfo=datetime.timezone.utc), 'atemp': 94044

('readings',
 [WeatherStationReading(station_id='EWXDAVIS01', station_type='DAVIS', request_id='a7410253-583c-4204-96bb-77bf6a2a14e7', request_datetime=datetime.datetime(2023, 8, 23, 18, 50, 10, 899160, tzinfo=datetime.timezone.utc), time_interval=None, data_datetime=datetime.datetime(2023, 8, 23, 17, 50, tzinfo=datetime.timezone.utc), atemp=940443648.89, pcpn=0.0, relh=80.0, lws0=None),
  WeatherStationReading(station_id='EWXDAVIS01', station_type='DAVIS', request_id='a7410253-583c-4204-96bb-77bf6a2a14e7', request_datetime=datetime.datetime(2023, 8, 23, 18, 50, 10, 899160, tzinfo=datetime.timezone.utc), time_interval=None, data_datetime=datetime.datetime(2023, 8, 23, 17, 55, tzinfo=datetime.timezone.utc), atemp=940443815.56, pcpn=0.0, relh=79.0, lws0=None),
  WeatherStationReading(station_id='EWXDAVIS01', station_type='DAVIS', request_id='a7410253-583c-4204-96bb-77bf6a2a14e7', request_datetime=datetime.datetime(2023, 8, 23, 18, 50, 10, 899160, tzinfo=datetime.timezone.utc), time_inter